# Caracteristica MOD13A1 NDVI y EVI google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='SATELLITE')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee,{"color" : "00FF11"},name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Coleccion MODIS13 A1

In [6]:
# Coleccion de MODIS LST
MODIS_Indice = ee.ImageCollection("MODIS/006/MOD13A1")\
              .filterDate('2019-01-01','2019-12-31')\
              .filterBounds(zona_ee)

In [7]:
# Conteo de imagenes
count = MODIS_Indice.size()
print("Cantidad de imagenes MODIS_Indice:", count.getInfo())

Cantidad de imagenes MODIS_Indice: 23


In [8]:
# Imprimir la lista de ID Imagenes
ID_MODIS_Indice = MODIS_Indice.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_MODIS_Indice)

['2019_01_01', '2019_01_17', '2019_02_02', '2019_02_18', '2019_03_06', '2019_03_22', '2019_04_07', '2019_04_23', '2019_05_09', '2019_05_25', '2019_06_10', '2019_06_26', '2019_07_12', '2019_07_28', '2019_08_13', '2019_08_29', '2019_09_14', '2019_09_30', '2019_10_16', '2019_11_01', '2019_11_17', '2019_12_03', '2019_12_19']


## Seleccion de imagen

In [9]:
ID_MODIS_Indice[14]

'2019_08_13'

In [10]:
# Importar imagen MODIS13 A1 LST
MODIS_img = ee.Image("MODIS/006/MOD13A1" + "/" + ID_MODIS_Indice[14])

In [11]:
# Nombre de las bandas
print(MODIS_img.bandNames().getInfo())

['NDVI', 'EVI', 'DetailedQA', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07', 'ViewZenith', 'SolarZenith', 'RelativeAzimuth', 'DayOfYear', 'SummaryQA']


## Seleccionar banda

In [12]:
# Seleccionar las bandas
MODIS_img = MODIS_img.select(["NDVI","EVI"])

In [13]:
# Nombre de las bandas
print(MODIS_img.bandNames().getInfo())

['NDVI', 'EVI']


## Reproyectar

In [14]:
# Reproyectar segun zona de estudio
MODIS_img_py = MODIS_img.reproject(crs = "EPSG:4326", scale = 500)

In [15]:
# Consultar Proyeccion
MODIS_img_py.projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:4326',
 'transform': [0.004491576420597608, 0, 0, 0, -0.004491576420597608, 0]}

## Factor de escala

In [16]:
# Multiplicar factor de escala
MODIS_img_py1 = MODIS_img_py.multiply(0.0001)

In [17]:
# Simbologia de MODIS
viz = {
    'min': 0,
    'max': 0.9,
    'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 
                'FCD163', '99B718', '74A901', '66A000',
                '529400', '3E8601', '207401', '056201',
                '004C00', '023B01', '012E01', '011D01', 
                '011301']
}

In [19]:
# Visualizar 
Map.addLayer(MODIS_img_py1.select(["NDVI"]), viz, name = "MODIS NDVI")
Map.addLayer(MODIS_img_py1.select(["EVI"]), viz, name = "MODIS EVI")
Map

Map(bottom=2507.0, center=[-13.838079936422462, -52.60253906250001], controls=(WidgetControl(options=['positio…

## Recortar zona estudio

In [20]:
# Recortar zona estudio
MODIS_img_py1_clip = MODIS_img_py1.clip(zona_ee)

In [21]:
# Visualizar 
Map.addLayer(MODIS_img_py1_clip.select(["NDVI"]), viz,name = "MODIS NDVI clip")
Map.addLayer(MODIS_img_py1_clip.select(["EVI"]), viz,name = "MODIS EVI clip")
Map

Map(bottom=2507.0, center=[-13.838079936422462, -52.60253906250001], controls=(WidgetControl(options=['positio…

## Reproyectar UTM

In [22]:
# Cambiar la proyeccion
MODIS_img_py1_clip_utm = MODIS_img_py1_clip.reproject(crs = "EPSG:32719", scale = 500)

## Exportar Imagen calibrado

In [23]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [24]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [25]:
# Exportar imagen directorio
geemap.ee_export_image(MODIS_img_py1_clip_utm, filename="MODIS_13_clip_utm.tif", scale=500, region=geometria, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE


In [26]:
# Exportar en google drive
geemap.ee_export_image_to_drive(MODIS_img_py1_clip_utm, description='MODIS_13_NDVI_EVI_clip_utm', folder='GEE_Python', region=geometria, scale=500)

Exporting MODIS_13_NDVI_EVI_clip_utm ...
